## *Passing Authority: Rekeying
#### 03.6 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2024-02-14 (started 2022-02-02)

- Rekey an account = change private key 

## Setup
Starting with chapter 3.4, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn, MultisigTransaction
import algosdk.error
import json

In [3]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI
ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU
BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU
CHARLS5INIT5KOLDK3F2QAZXY7N4W7GOE6UTXHEHS3RBNCR2WJXRRTH3GY
DINAAZWBJM6DRY73CWK7IOA4S7C6PVXQMQ4DJ3TK3DAGAGFJTX5KEYESDI


### Connecting to the Algorand testnet

In [4]:
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['api_token'])
algod_client.status()["last-round"]

38992391

## Rekeying
Case 1
* Imagine your private key has been compromised
* You have lots of assets and maybe ongoing smart contracts with this address
* So you want to "change the password"

Case 2
* Bob is very busy and wants Charlie to sign for him

#### Bob rekeys to Charlie
Rekeying is done via a simple payment transaction of 0 ALGOs, that has an additional rekey entry

In [5]:
# Step 1: prepare special transaction
sp = algod_client.suggested_params()
amt = 0                                          # <-- Send zero ALGOs

txn = PaymentTxn(sender = Bob['public'],         # <-- We rekey the Bob account
                 sp = sp, 
                 receiver = Bob['public'],       # <-- (any recipeint, as it is a zero-amount transaction)
                 amt = amt,
                 rekey_to = Charlie['public']    # <-- Charlie should be able to sign for Bob
                )

In [6]:
# Steps 2+3: sign and send
stxn = txn.sign(Bob['private'])                 # <-- Bob signs one last time
txid = algod_client.send_transaction(stxn)

In [7]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  38992393.
Waiting for round 38992393 to finish.
Waiting for round 38992394 to finish.
Transaction JHIMBOAITDLAFNNB3XQVZYJPDFGGMODQ72KKJPGBB2O5HY6KZY4A confirmed in round 38992395.


#### Charlie signs for Bob
* Charlie makes a payment of 1.4 ALGO to Alice out of Bob's account

In [8]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(1.4*1E6)                                

txn = PaymentTxn(sender = Bob['public'],           # <-- From Bob
                 sp = sp, 
                 receiver = Alice['public'],       # <-- To Alice 
                 amt = amt
                 )

# Steps 2+3: sign and send
stxn = txn.sign(Charlie['private'])                # <-- Charlie now signs for Bob
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  38992395.
Waiting for round 38992395 to finish.
Waiting for round 38992396 to finish.
Transaction T6EFTZLNZBGRXY4AIMUQT2PUESQMNJR4YX2AKYI5ZIGVOEAYZWLQ confirmed in round 38992397.


#### Bob cannot sign for himself
* Bob wants to make a payment of 1.7 ALGO to Dina

In [9]:
# Step 1: prepare special transaction

sp = algod_client.suggested_params()
amt = int(1.7*1E6)                                

txn = PaymentTxn(sender = Bob['public'],         # <-- From Bob
                 sp = sp, 
                 receiver = Dina['public'],       # <-- To Dina 
                 amt = amt
                )

# Steps 2+3: sign and send
stxn = txn.sign(Bob['private'])                 # <-- Bob tries to sign
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

AlgodHTTPError: TransactionPool.Remember: transaction VSPDXCX5IDLQMGJ7HCDVAOINT7WJEJMK6JFHWYOOCH7NJBHJWI5Q: should have been authorized by CHARLS5INIT5KOLDK3F2QAZXY7N4W7GOE6UTXHEHS3RBNCR2WJXRRTH3GY but was actually authorized by BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU

#### ❗️Charlie rekeys back to Bob
* Charlie rekeys the Bob account back to Bob
* **Important step for the rest of the course**

In [10]:
# Step 1: prepare special transaction
sp = algod_client.suggested_params()
amt = 0                                          # <-- Rekeying = send 0 ALGOs

txn = PaymentTxn(sender = Bob['public'],         # <- We rekey the Bob account
                 sp = sp, 
                 receiver = Dina['public'],      # <-- (any recipeint, as it is a zero-amount transaction)
                 amt = amt,
                 rekey_to = Bob['public']        # <-- Bob should (again) be able to sign for himself
                )

In [11]:
# Steps 2+3: sign and send
stxn = txn.sign(Charlie['private'])              # <-- Charlie signs the rekeying (one last time)
txid = algod_client.send_transaction(stxn)

In [12]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  38992404.
Waiting for round 38992404 to finish.
Waiting for round 38992405 to finish.
Transaction W3V75ZPYPRC6NUH6OKLOUNUIW72YDUWQN2CVH7X4VBQEQSWXTWLQ confirmed in round 38992406.


#### Exercise
* To test the rekeying, run the payment transactions above again
    * Charlie cannot sign for Bob anymore
    * Bob can sign again for himself

## ❗️Things that can and will go wrong
* Be sure if you trust the new signature holder
* You cannot rekey back to yourself
* **Rekeying can be easily slipped into a payment transaction**
* Wrong address or typo in rekey means you loose all your money
    * The protocol does not check whether you or someone has actually access to the rekey address